In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_area(file_path):
        
    image = cv2.imread(file_path)

    above_mask = draw_above_mask(image,file_path)
    image_crop=image[:,:above_mask.shape[1]]
#     above_mask = above_mask[:,:rightmost_x]
    # 将图像转换为HSV颜色空间
    hsv = cv2.cvtColor(image_crop, cv2.COLOR_BGR2HSV)

    # 定义深蓝色范围
    lower_dark_blue = np.array([110, 50, 50])  # 深蓝色下限
    upper_dark_blue = np.array([130, 255, 255])  # 深蓝色上限

    # 创建深蓝色掩码
    dark_blue_mask = cv2.inRange(hsv, lower_dark_blue, upper_dark_blue)
    
    # 与深蓝色区域的掩膜进行按位与操作，获取线上方的深蓝色区域
    blue_area_above_line = cv2.bitwise_and(above_mask, dark_blue_mask)
    
    # 进行膨胀和腐蚀操作，消除噪声
    kernel = np.ones((5, 5), np.uint8)
    blue_area_above_line = cv2.morphologyEx(blue_area_above_line, cv2.MORPH_OPEN, kernel)

    # 计算非零像素数量
    blue_area = cv2.countNonZero(blue_area_above_line)
    if blue_area>0:
        # 获取非零像素的列索引
        non_zero_columns = np.nonzero(blue_area_above_line)[1]

        # 获取最左边的横坐标
        leftmost_x = np.min(non_zero_columns)

        start_coor = leftmost_x - 436
    else:
        start_coor = 0

    return blue_area, start_coor

In [2]:
import numpy as np

def draw_above_mask(image,path):
    
    filename = path.split("/")[-1].split(".")[0]
    case_number = filename.split("-")[0]
    
#     print("filename",filename)
    # 读取.dat文件，跳过前65行，从第66行开始读取数据
    data = pd.read_csv("/kaggle/input/airfoil-geo/"+case_number+".dat", header=None, delimiter='\s+', skiprows=34, nrows=25)
    data = pd.read_csv("/kaggle/input/airfoil-geo/"+case_number+".dat", header=None, delimiter='\s+', skiprows=91, nrows=25)
    data = pd.read_csv("/kaggle/input/airfoil-geo/"+case_number+".dat", header=None, delimiter='\s+', skiprows=65, nrows=25)
    airfoil_coor=data.values
    
    scale = [[1190/3,1190/3] for _ in range(len(airfoil_coor))]

    airfoil_coor = [[int(x * y ) for x, y in zip(sub_a, sub_b)] for sub_a, sub_b in zip(airfoil_coor, scale)]
    
    airfoil_coor = [[x + 436, y + 472] for x, y in airfoil_coor]
    
    # 假设你有一条线的坐标点组成的数组 line_points

    # 找到线的最左边的点
    leftmost_point = min(airfoil_coor, key=lambda point: point[0])

    # 计算对称线的 y 坐标
    y_left = leftmost_point[1]
    symmetric_line = [[point[0], 2 * y_left - point[1]] for point in airfoil_coor]

    symmetric_line.append([symmetric_line[-1][0],0])
    symmetric_line.append([symmetric_line[0][0],0])

    # 创建一个与画布相同大小的掩膜图像
    line_mask = np.zeros_like(image[:, :, 2])

    # 将线的坐标数据绘制在掩膜图像上
    cv2.fillPoly(line_mask, [np.array(symmetric_line)], 255)
        
    line_mask = line_mask[:,:int((data.values[0][0]+1)*1190/3)+44]
    
    # 创建一个与画布相同大小的掩膜图像
    line_mask2 = np.zeros_like(image[:, :, 2])

    # 将线的坐标数据绘制在掩膜图像上
    cv2.polylines(line_mask2, [np.array(symmetric_line)], False, 255, thickness=2)

    return line_mask

In [3]:
# capture_airfoil("/kaggle/input/flowfieldpic/Pic/case411-175.png")

calculate_area("/kaggle/input/flowfieldpic/Pic/case419-235.png")

(1594, 217)

In [4]:
import os
import glob

# 指定文件夹路径
pic_path = glob.glob("/kaggle/input/flowfieldpic/Pic/*")

df = pd.DataFrame(columns=["图片名", '失速面积','失速开始位置'])

In [5]:
# 遍历文件夹中的所有文件
for file_path in pic_path:
    stall_area,start_coor=calculate_area(file_path)
    # 迭代添加行数据
    data = {'图片名': file_path,
            '失速面积': stall_area,
           '失速开始位置':start_coor}
    df.loc[len(df)] = data
#     print("失速面积,失速开始位置", stall_area,start_coor)
    df.to_excel('分类结果.xlsx', index=False)
    

In [6]:
print(df)
df.to_excel('分类结果.xlsx', index=False)

                                                  图片名  失速面积  失速开始位置
0      /kaggle/input/flowfieldpic/Pic/case174-170.png     0       0
1      /kaggle/input/flowfieldpic/Pic/case181-130.png     0       0
2      /kaggle/input/flowfieldpic/Pic/case289-170.png  4521     194
3      /kaggle/input/flowfieldpic/Pic/case462-205.png  5623     165
4      /kaggle/input/flowfieldpic/Pic/case306-145.png     0       0
...                                               ...   ...     ...
14495  /kaggle/input/flowfieldpic/Pic/case296-200.png     0       0
14496    /kaggle/input/flowfieldpic/Pic/case7-230.png  2029     236
14497  /kaggle/input/flowfieldpic/Pic/case279-230.png  8566      97
14498  /kaggle/input/flowfieldpic/Pic/case294-225.png  3833     225
14499  /kaggle/input/flowfieldpic/Pic/case259-240.png   920     235

[14500 rows x 3 columns]


In [7]:
def capture_airfoil(file_path):

    image = cv2.imread(file_path)
    # 获取图像的高度和宽度
    height, width = image.shape[:2]
    print("height, width:",height, width)

    # 计算要保留的像素范围
    top_h=int(0.15*height)
    bottom_h=int(0.85*height)

    top_w=int(0.15*width)
    bottom_w=int(0.85*width)

    # 截取图像的上部分
    image = image[top_h:bottom_h, top_w:bottom_w]
    height, width = image.shape[:2]

    # 将图像转换为灰度图像
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 创建一个新的图形
    plt.figure()

    # 将图像显示在图形中
    plt.imshow(gray)

    # 显示图形
    plt.show()

    # 对灰度图像进行阈值处理
    _, threshold = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
    
    # 创建一个新的图形
    plt.figure()

    # 将图像显示在图形中
    plt.imshow(threshold)

    # 显示图形
    plt.show()
    
    # 进行膨胀和腐蚀操作，消除噪声
    kernel = np.ones((5, 5), np.uint8)
    threshold = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)
    
    # 连通组件分析
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(threshold)

    # 获取每个区域掩膜的单独最大和最小纵坐标
    for i in range(1, num_labels):  # 跳过背景区域（标签0）
        min_y = stats[i, 1]
        max_y = stats[i, 1] + stats[i, 3]
        if(min_y > 0 and max_y < height):
            region_mask = np.uint8(labels == i) * 255  # 创建区域掩膜
            region_image = cv2.bitwise_and(image, image, mask=region_mask)  # 通过按位与操作提取区域图像    
            # 创建一个新的图形
            plt.figure()

            # 将图像显示在图形中
            plt.imshow(region_image)

            # 显示图形
            plt.show()
            
            gray_mask = cv2.cvtColor(region_image, cv2.COLOR_BGR2GRAY)
            
            # 获取非零像素坐标
            non_zero_pixels = cv2.findNonZero(gray_mask)

            # 获取最左边和最下边的坐标
            left_x = np.min(non_zero_pixels[:,:,0])
            bottom_y = np.max(non_zero_pixels[:,:,1])

            print("机翼最左下坐标：({}, {})".format(left_x, bottom_y))

    # 使用Canny边缘检测
    edges = cv2.Canny(region_image, 100, 200)

    # 寻找最外侧的轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return calculate_area(contours,image,file_path)